In [1]:
!pip install keras-tuner
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch, Hyperband
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') / 255
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 30.4 MB/s eta 0:00:00
Using TensorFlow backend
11490434/11490434 [==============================] - 0s 0us/step


<ipython-input-1-a4263ea13338>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch, Hyperband


In [2]:
def build_model(hp):
 model = Sequential()
 hp_filters = hp.Int('filters', min_value=32, max_value=128, step=32)
 model.add(Conv2D(filters=hp_filters, kernel_size=5, activation='relu',padding='same', input_shape=(28, 28, 1)))
 model.add(MaxPooling2D(2))
 hp_filters1 = hp.Int('filters1', min_value=32, max_value=128, step=32)
 model.add(Conv2D(filters=hp_filters1, kernel_size=5, activation='relu', padding='same', input_shape=(28, 28, 1)))
 model.add(MaxPooling2D(2))
 hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
 model.add(Dropout(rate=hp_dropout))
 model.add(Flatten())
 hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
 model.add(Dense(units=hp_units, activation='relu'))
 model.add(Dense(10, activation='softmax'))
 hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
 model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate),
 loss='categorical_crossentropy',
 metrics=['accuracy'])
 return model

In [3]:
#tuner = RandomSearch(
 #build_model,
 #objective='val_accuracy',
 #max_trials=12,
 #directory='my_dir',
 #project_name='Assignment1_RandomSearch7')

tuner = Hyperband(
 build_model,
 objective='val_accuracy',
 max_epochs=20, # It's better to set max_epochs larger for Hyperband
 factor=3,
 directory='my_dir',
 project_name='hyperband')

tuner.search(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Trial 30 Complete [00h 03m 19s]
val_accuracy: 0.9939000010490417

Best val_accuracy So Far: 0.9940999746322632
Total elapsed time: 00h 42m 57s


In [4]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
filters1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [5]:
best_model = tuner.get_best_models(num_models=1)[0]

In [6]:
#For Test Accuracy
evaluation = best_model.evaluate(test_images, test_labels)
print("Test accuracy:", evaluation[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.0314 - accuracy: 0.9941
Test accuracy: 0.9940999746322632


In [7]:
best_hyperparameters=tuner.get_best_hyperparameters()[0]

In [8]:
hp_filters = best_hyperparameters.get('filters')
hp_filters1 = best_hyperparameters.get('filters1')
hp_dropout = best_hyperparameters.get('dropout')
hp_units = best_hyperparameters.get('units')
hp_learning_rate = best_hyperparameters.get('learning_rate')

In [9]:
print(f"Best 'filters' hyperparameter: {hp_filters}")
print(f"Best 'filters' hyperparameter: {hp_filters1}")
print(f"Best 'dropout' hyperparameter: {hp_dropout}")
print(f"Best 'units' hyperparameter: {hp_units}")
print(f"Best 'learning_rate' hyperparameter: {hp_learning_rate}")

Best 'filters' hyperparameter: 96
Best 'filters' hyperparameter: 32
Best 'dropout' hyperparameter: 0.30000000000000004
Best 'units' hyperparameter: 128
Best 'learning_rate' hyperparameter: 0.001
